In [17]:
import numpy as np
import tt
import tt.cross.rectcross as rect_cross
#import rect_cross
import time
import scipy.special

## Problem setting

This code solves multicomponent Smoluchowski equation with source and sink terms


$$\frac{\partial n(\overline{v}, t)}{\partial t} = \frac{1}{2}\int_0^{v_1} ... \int_0^{v_d} K(\overline{v}-\overline{u}; \overline{u}) n(\overline{v} - \overline{u}, t) n(\overline{u}, t) du_1 ... du_d - \\
\\ - n(\overline{v},t) \int_0^{V_{\max}} ... \int_0^{V_{\max}} K(\overline{u}; \overline{v}) n(\overline{u},t) du_1 ... du_d ~ + ~ q(v_1, \ldots, v_d)$$

details about the numerical scheme can be found in papers:$$\\$$
1. Matveev SA, DA Zheltkov, EE Tyrtyshnikov, AP Smirnov, $$\\$$
Tensor train versus Monte Carlo for the multicomponent Smoluchowski coagulation equation, Journal of Computational Physics (2016) 316, 164-179$$\\$$

2. Smirnov AP, Matveev SA, Zheltkov DA, Tyrtyshnikov EE, $$\\$$
Fast and accurate finite-difference method solving multicomponent Smoluchowski coagulation equation with source and sink terms, Procedia Computer Science  (2016) pp. 2141-2146

## Setup the parameters

In [18]:
d = 2
N = 800
r = 1
h = 0.125
tau = 0.025
N_steps = 10
tolerance = 1e-6
print_res = 0
T = N_steps * tau
check_error = 0

In [19]:
def Coag_K(x):#Coagulation kernel
    if (x.size % 2) != 0:
        print "Kernel must depend on even number of indeces!"
        exit()
    
    u = (x[:, :d ] + 1e-2) * h
    v = (x[:, d: ] + 1e-2) * h
    # Ballistic kernel
    # (u^(1/3) + v^(1/3))^2 * sqrt(1/u + 1/v)
    return (np.power(u.sum(axis = 1), 1.0 / 3.0) + np.power(v.sum(axis = 1), 1.0 / 3.0))**2.0 *  np.power( 1.0 / u.sum(axis = 1) + 1.0 / v.sum(axis = 1), 0.5)
    # Generalized multiplication a = 0.1
    #return np.power(u.sum(axis = 1), 0.1) * np.power(v.sum(axis = 1), -0.1) + np.power(u.sum(axis = 1), -0.1) * np.power(v.sum(axis = 1), 0.1)

def q(x):#Source
    return np.exp(-x.sum(axis = 1) * h)

In [20]:
def Check_n(x):
    return np.exp(-x.sum(axis = 1) * h)

def Check_Mass(x):
    return (x.sum(axis = 1)) * h

def Start_Cond(x):
    return np.exp(-x.sum(axis = 1) * h)

def Analytical(x):
    return np.exp(-h * x.sum(axis=1)) / (1.0 + T / 2.0)**2 * scipy.special.i0(  2.0 * np.sqrt( h**d * np.prod(x, axis = 1)  * T / (2.0 + T)))

## TT-representation of operators

the following cell contains implementation of TT-integration via trapezoids rule for the following integrals:
    
$$\int_0^{v_1} ... \int_0^{v_d} K(\overline{v}-\overline{u}; \overline{u}) n(\overline{v}) du_1 ... du_d$$

How to perform its simplified version:
    
\begin{align*}\notag
&q(v_1, \ldots, v_d) =\int_0^{v_1} \ldots\int_0^{v_d} f(v_1 - u_1, \ldots, v_d - u_d) g(u_1, \ldots, u_d) du_1 \ldots du_d \\ \notag
 &=\sum_{\alpha_0, \ldots, \alpha_d} \sum_{\beta_0, \ldots, \beta_d} \int_0^{v_1} \ldots\int_0^{v_d} f_1(\alpha_0, v_1 - u_1, \alpha_1) \ldots f_d(\alpha_{d-1}, v_d - u_d, \alpha_d) g_1(\beta_1, u_1, \beta_2) \ldots g_d(\beta_{d-1}, u_d, \beta_d) du_1 \ldots du_d\\ \notag
&=\sum_{\alpha_1, \ldots, \alpha_d} \sum_{\beta_0, \ldots, \beta_d} \int_0^{v_1} f_1(\alpha_0, v_1 - u_1, \alpha_1)\ g_1(\beta_1, u_1, \beta_2) du_1 \ldots \int_0^{v_d}  f_d(\alpha_{d-1}, v_d - u_d, \alpha_d)  \ldots g_d(\beta_{d-1}, u_d, \beta_d) du_d.
\end{align*}

One-dimensional convolutions will stand as:

\begin{equation}\notag
 h \begin{bmatrix}
\frac{1}{2}f_k(\alpha_{k-1}, v_{k_0}, \alpha_k) &  0  & 0 & 0 & 0 \\ 
f_k(\alpha_{k-1}, v_{k_1}, \alpha_k) &  \frac{1}{2}f_k(\alpha_{k-1}, v_{k_0}, \alpha_k) & 0 & 0 & 0 \\ 
f_k(\alpha_{k-1}, v_{k_2}, \alpha_k) &  f(\alpha_{k-1}, v_1, \alpha_k) & \frac{1}{2}f_k(\alpha_{k-1}, v_{k_0}, \alpha_k) & 0 & 0 \\ 
\ldots & \ldots & \ldots& \ldots & \ldots\\ 
f_k(\alpha_{k-1}, v_{k_N}, \alpha_k) &  f_k(\alpha_{k-1}, v_{k_{N-1}}, \alpha_k)& \ldots & \ldots & \frac{1}{2}f_k(\alpha_{k-1}, v_{k_0}, \alpha_k)\\
\end{bmatrix} \times 
\begin{bmatrix}
\frac{1}{2}g_k(\beta_{k-1}, v_{k_0}, \beta_k)  \\ 
g_k(\beta_{k-1}, v_{k_1}, \beta_k) \\ 
g_k(\beta_{k-1}, v_{k_2}, \beta_k)  \\ 
\ldots \\ 
g_k(\beta_{k-1}, v_{k_N}, \beta_k) \\
\end{bmatrix}.
\end{equation}

In [21]:
def First_Integral(TT_solution, Coag_Kernel):
    carriages = tt.tensor.to_list(Coag_Kernel)
    
    #list of last d - 1 carriages
    list_u = carriages[d + 1  :]
    #list of first d - 1 carriages
    list_v = carriages[ : d - 1]
    
    for alpha in xrange(Coag_Kernel.r[d]):#Loop along the middle index of kernel
        #print 'alpha', alpha
        #insert one slice from d+1-th carriage and create TT
        list_u.insert(0, carriages[d] [alpha : alpha + 1, :, :])
        #append one slice from d-th carriage and create TT
        list_v.append(carriages[d - 1][:, :, alpha : alpha + 1])
        #get tensor trains from the lists
        K_u_part = tt.tensor.from_list(list_u)
        K_v_part = tt.tensor.from_list(list_v)
        
        #compute Integrands  
        
        K_u_part = tt.multifuncrs([K_u_part, TT_solution], lambda x: x.prod(axis = 1), eps = tolerance) # K_alpha (u) * n(u)
        K_v_part = tt.multifuncrs([K_v_part, TT_solution], lambda x: x.prod(axis = 1), eps = tolerance) # K_alpha(v) * n(v)
        K_u_part = tt.tensor.to_list(K_u_part)
        K_v_part = tt.tensor.to_list(K_v_part)
        
        for i in xrange(d):
            # add the required zeros, multiply by the integration mesh step
            K_u_part[i][:, 0, :] = K_u_part[i][:, 0, :] / 2.0
            K_v_part[i][:, 0, :] = K_v_part[i][:, 0, :] / 2.0
            K_u_part[i] = np.concatenate( (K_u_part[i], np.zeros([K_u_part[i].shape[0], N - 1, K_u_part[i].shape[2]])), axis = 1)
            # do the first FFT
            K_u_part[i] = np.fft.fft(K_u_part[i], axis = 1, n = 2 * N - 1)
            # add the required zeros and multiply by the integration mesh step
            K_v_part[i] = np.concatenate( (K_v_part[i], np.zeros([K_v_part[i].shape[0], N - 1, K_v_part[i].shape[2]])), axis = 1)
            # do the first FFT
            K_v_part[i] = np.fft.fft(K_v_part[i], axis = 1, n = 2 * N - 1)

        # Build tensor trains after the axis-wide FFTs
        K_u_part = tt.tensor.from_list(K_u_part)
        K_v_part = tt.tensor.from_list(K_v_part)
        
        
        # Elementwise product of 2 TTs; Store the results in K_u_part(allows us not to make new TT but replase the old one)
        K_u_part = tt.multifuncrs([K_u_part, K_v_part], lambda x: x.prod(axis = 1), eps = tolerance)
        
        # Back to list of TT-cores
        K_u_part = tt.tensor.to_list(K_u_part)

        for i in xrange(d):
            # Inverse FFT along each phase-spase axis
            K_u_part[i] = np.fft.ifft(K_u_part[i], axis = 1, n = 2 * N - 1)
            # Truncation. Here we delete the additional (n-1)-zeros
            K_u_part[i] = h *  np.abs(K_u_part[i][:, : N, :]) * np.sign(K_u_part[i][:, : N, :])
            K_u_part[i][:,0,:] *= 0.0;
        # Restore the TT
        K_u_part = tt.tensor.from_list(K_u_part)
        # Save the results in I1
        if alpha == 0:
            I1 = K_u_part
        else:
            I1 = tt.multifuncrs([I1, K_u_part], lambda x: x.sum(axis = 1), eps = tolerance)
        #delete used slices before addition of the new ones
        list_v.pop(d - 1)
        list_u.pop(0)
        #print 'I1'
        #print I1
    #============================================================================================
    I1 = tt.multifuncrs([I1], lambda x: x.real, eps = tolerance)
    return I1

## More integrals
The following cell contains implementation of TT-integration via trapezoids rule for the following integrals
$$q(\overline{v}) = \int_0^{V_{\max}} ... \int_0^{V_{\max}} K(\overline{u}; \overline{v}) f(\overline{u}) du_1 ... du_d$$

$$\\$$
How to perform it:
\begin{align*}
& q(\overline{v}) = \int_0^{V_{\max}} ... \int_0^{V_{\max}} K(\overline{u}; \overline{v}) f(\overline{u}) du_1 ... du_d = \sum_{\overline{\alpha}, \overline{\beta}} K_1^v(\alpha_0, v_1, \alpha_1) \ldots K_d^v(\alpha_{d-1}, v_d, \alpha_{d}) 
\times 
\\ & \times \int_0^{V_{\max}} K_1^u(\alpha_{d}, u_1, \alpha_{d+1}) f_1(\beta_0, u_1, \beta_1) du_1 \ldots \int_0^{V_{\max}} K_d^u(\alpha_{2d-1}, u_d, \alpha_{2d}) f_d(\beta_{d-1}, u_d, \beta_d) du_d = 
\\ & \sum_{\overline{\alpha}} K_1^v(\alpha_0, v_1, \alpha_1) \ldots K_d^v(\alpha_{d-1}, v_d, \alpha_{d}) \sum_{\overline{\beta}} \int_0^{V_{\max}} K_1^u(\alpha_{d}, u_1, \alpha_{d+1}) f_1(\beta_0, u_1, \beta_1) du_1 \times 
\\ &\ldots  \times \int_0^{V_{\max}} K_d^u(\alpha_{2d-1}, u_d, \alpha_{2d}) f_d(\beta_{d-1}, u_d, \beta_d) du_d = \sum_{\alpha_d = 1}^{R_d} \widetilde{K^v_{\alpha_d}}(\overline{v}) I_{\alpha_d},
\end{align*}

In [15]:
def Second_Integral(TT_solution, Coag_Kernel):
    # create weights of rectangular quadrature like tt.ones * h**d;
    # we multiply each carriage by h to avoid the machine zero
    weights = tt.ones(N, d)
    carriages = tt.tensor.to_list(weights)
    for i in xrange(d):
        carriages[i] = carriages[i] * h
        carriages[i][:,0,:] /= 2
        carriages[i][:,N-1,:] /= 2
    weights = tt.tensor.from_list(carriages)
    #create list of carriages of coagulation kernel

    carriages = tt.tensor.to_list(Coag_Kernel)

    #list of first d carriages
    list_v = carriages[ : d]
    #list if last d - 1 carriages
    list_u = carriages[ d + 1 :]
    Integral = np.zeros(Coag_Kernel.r[d])
    #print 'Coagulation kernel in TT format'
    #print Coag_Kernel 
    #print '============================================================'
    for alpha in xrange(carriages[d - 1 ].shape[2]):
        #print 'alpha', alpha
        #insert one slice from d+1-th carriage to create TT
        list_u.insert(0, carriages[d] [alpha : alpha + 1, :, :])
        #get tensor trains from the lists
        #K_v_part = tt.tensor.from_list(list_v)
        K_u_part = tt.tensor.from_list(list_u)
        #compute Integrand (u) = K_u(u) * n(u) 
        K_u_part = tt.multifuncrs([K_u_part, TT_solution], lambda x: np.prod(x, axis=1), eps = tolerance)
        #compute I[alpha] = \int_0^{v_{max}} Integrand[alpha](u) du will be a vector of size r_d
        # sum of results over the middle d-th index
        Integral[alpha] = tt.dot(K_u_part, weights)
        list_u.pop(0)
    #============================================================================================
    list_v[d - 1] = list_v[d - 1].reshape(list_v[d - 1].shape[0] * list_v[d - 1].shape[1], list_v[d - 1].shape[2])
    list_v[d - 1] = list_v[d - 1].dot(Integral)
    list_v[d - 1] = list_v[d - 1].reshape(carriages[d - 1].shape[0], carriages[d - 1].shape[1], 1)
    I2 = tt.tensor.from_list(list_v)
    I2 = tt.multifuncrs([I2, TT_solution], lambda x: np.prod(x, axis = 1), eps = tolerance)

    #print '============================================================'
    #print 'Coagulation Kernel'
    #print Coag_Kernel
    return I2

## Finally

The last cell contains implementation of predictor-corrector scheme solving the Cauchy problem for Smoluchowski coagulation equation
$$
\begin{matrix}
n^{k+\frac{1}{2}}(\overline{i})  = 
\frac{\tau}{2} \left( L_{1}(\overline{i}) (n^{k}) - n^{k}(\overline{i}) 
L_{2}(\overline{i}) (n^{k}) + q^k(\overline{i})  \right) + n^{k}(\overline{i})  \\
n^{k+1 }(\overline{i}) = \tau \left( L_1(\overline{i}) (n^{k+\frac{1}{2}}) - 
                                    n^{k+\frac{1}{2}}(\overline{i}) L_{2}(\overline{i}) (n^{k+\frac{1}{2}}) + 
                                    q^{k+\frac{1}{2}}(\overline{i})\right) + n^{k}(\overline{i}).
\end{matrix}
$$

In [16]:
#approximate starting condition
print 'Approximate starting condition'
x0 = tt.rand(N, d, r)
TT_Solution = rect_cross.cross(Start_Cond, x0, nswp = 6, kickrank = 1, rf = 2)
#TT_Solution = TT_Solution.round(tolerance)
print 'Approximate TT for check of total mass'
Check_mass  = rect_cross.cross(Check_Mass, x0, nswp = 6, kickrank = 1, rf = 2)
#TT_Analyt   = rect_cross.cross(Analytical, x0, nswp = 6, kickrank = 1, rf = 2)
#print TT_Analyt

print 'Approximate source'
source = rect_cross.cross(q, x0, nswp = 6, kickrank = 1, rf = 2)
weights = tt.ones(N, d)
carriages = tt.tensor.to_list(weights)
for i in xrange(d):
    carriages[i] = carriages[i] * h
    carriages[i][:, 0, :] /= 2
    carriages[i][:, N - 1, :] /= 2
weights = tt.tensor.from_list(carriages)


Mass_tt = tt.multifuncrs([TT_Solution, Check_mass], lambda x: np.prod(x, axis = 1 ), eps = tolerance)
#Mass_tt = Mass_tt.round(tolerance/1e2)
print 'Starting mass = ', tt.dot(Mass_tt, weights)
print 'Approximate kernel'
#test with ballistic kernel
#
#x0 = tt.rand(N, 2 * d, r)
#Coag_Kernel = rect_cross.cross(Coag_K, x0, nswp = 6, kickrank = 1, rf = 2)
#print Coag_Kernel
#print '============================================================'
#
#test with K(u,v) = 1
#
Coag_Kernel = tt.ones(N, 2 * d)
#Coag_Kernel = Coag_Kernel.round(tolerance)

t1 = time.clock()
print 'alpha = ', Coag_Kernel.r[d]
density_file = open('density_time.log', 'w')
for t in xrange(N_steps):
    print '=============================================================='
    print 'Step', t + 1
    print '=============================================================='

    First_integral = 0.25 * tau * First_Integral(TT_Solution, Coag_Kernel)

    Second_integral = - tau * 0.5 * Second_Integral(TT_Solution, Coag_Kernel)
    print 'First integral predictor'
    print First_integral
    print 'Second integral predictor'
    print Second_integral

    TT_Solution_predictor = tt.multifuncrs([TT_Solution, First_integral, Second_integral, 0.5 * tau * source], lambda x: np.sum(x, axis = 1) , eps = tolerance)

    First_integral = 0.5 * tau * First_Integral(TT_Solution_predictor, Coag_Kernel)

    Second_integral = - tau * Second_Integral(TT_Solution_predictor, Coag_Kernel)

    print 'First integral corrector'
    print First_integral
    print 'Second integral corrector'
    print Second_integral

    TT_Solution = tt.multifuncrs([TT_Solution, First_integral, Second_integral, tau * source], lambda x:  np.sum(x, axis = 1), eps = tolerance)

    Mass_tt = tt.multifuncrs([TT_Solution, Check_mass], lambda x: np.prod(x, axis = 1), eps = tolerance)
    print 'mass = ', tt.dot(Mass_tt, weights)
    print 'density =', tt.dot(TT_Solution, weights)
    density_file.write("%f %f\n"%((t + 1) * tau, tt.dot(TT_Solution, weights)))
    print 'Solution'
    print TT_Solution
    print '=============================================================='

    if print_res and (N <= 20000):
        print 'saving results into file %s.dat'%(t)
        res = open("%s.dat"%(t), 'w')
        for x in xrange(N):
            res.writelines("\n")
            for y in xrange(N):
                #print x*h, ' ', y*h, ' ', (x + y) * h * TT_Solution[x,y]
                res.writelines("%s %s %s\n"%(y * h, x * h, np.abs((x + y) * h * TT_Solution[y,x].real)))
        res.close()
t2 = time.clock()
print 'time = ', t2 - t1


Approximate starting condition
swp: 0/5 er_rel = 1.7e+02 er_abs = 7.9e+02 erank = 3.0 fun_eval: 6400
swp: 1/5 er_rel = 3.5e-16 er_abs = 1.6e-15 erank = 5.0 fun_eval: 19200
Approximate TT for check of total mass
swp: 0/5 er_rel = 1.0e+00 er_abs = 8.6e+04 erank = 3.0 fun_eval: 6400
swp: 1/5 er_rel = 2.4e-12 er_abs = 2.1e-07 erank = 5.0 fun_eval: 19200
Approximate source
swp: 0/5 er_rel = 1.7e+02 er_abs = 7.9e+02 erank = 3.0 fun_eval: 6400
swp: 1/5 er_rel = 3.5e-16 er_abs = 1.6e-15 erank = 5.0 fun_eval: 19200
=multifuncrs= sweep 1{2}, max_dy: 1.136e+13, erank: 1.41421
=multifuncrs= sweep 2{1}, max_dy: 4.056e-14, erank: 2
=multifuncrs= sweep 2{2}, max_dy: 4.056e-14, erank: 2.44949
=multifuncrs= sweep 3{1}, max_dy: 0.000e+00, erank: 1.41421
Starting mass =  1.99999796802
Approximate kernel
alpha =  1
Step 1
=multifuncrs= sweep 1{2}, max_dy: 4.419e+36, erank: 1.41421
=multifuncrs= sweep 2{1}, max_dy: 3.107e-16, erank: 1.73205
=multifuncrs= sweep 2{2}, max_dy: 3.425e-16, erank: 2
=multifuncrs